# Reading ModelSEED Biochemistry from ModelSEED Database

This notebook provides examples to read data from the biochemistry database

The biochemistry database is represented by the `modelseedpy.biochem.modelseed_biochem.ModelSEEDDatabase` class

The database can be instantiated from either a local github copy of the database repository or loaded via web url from github.
* `from_local` - from_local(*path to repository*)
    * Example:
        1. first we obtain a copy of the repository: `git clone https://github.com/ModelSEED/ModelSEEDDatabase.git`
        2. then we can load the repository with `from_local`: `from_local('/home/user/ModelSEEDDatabase')`
* `from_github` - from_github(*commit version*)
    * Example:
        * load the dev branch from github (default: https://github.com/ModelSEED/ModelSEEDDatabase): `from_github('dev')`
        * load a specific commit version from github: `from_github('194ac8afe48f8a606c0dd07ba3c7af10c02ba2fd')`
        * load from another fork/repository: `from_github('master', 'https://raw.githubusercontent.com/Fxe/ModelSEEDDatabase')`

In [1]:
import logging
import pandas as pd
from modelseedpy.biochem.modelseed_biochem import get_structures_from_df, get_aliases_from_df, get_names_from_df, process_aliases, load_metabolites_from_df
from modelseedpy.biochem.modelseed_biochem import ALIAS_RXN_IDENTIFIERS_ORG, ALIAS_MODELS
from modelseedpy.biochem.modelseed_biochem import from_local2, from_local, from_github
from modelseedpy.biochem.modelseed_reaction import ModelSEEDReaction, ModelSEEDReaction2
logger = logging.getLogger(__name__)

In [2]:
%%time
modelseed_local2 = from_local2('/home/fliu/workspace/python/ModelSEEDDatabase/')

CPU times: user 16.8 s, sys: 668 ms, total: 17.5 s
Wall time: 17.5 s


In [3]:
%%time
modelseed_local1 = from_local('/home/fliu/workspace/python/ModelSEEDDatabase/')

CPU times: user 42.2 s, sys: 503 ms, total: 42.7 s
Wall time: 42.7 s


In [4]:
%%time
modelseed_git = from_github('dev')

CPU times: user 18.8 s, sys: 1.12 s, total: 19.9 s
Wall time: 24.3 s


In [5]:
inchi_key_lookup = {}
for cpd in modelseed_git.compounds:
    inchi_key = cpd.inchi_key
    inchi_key = None if pd.isna(inchi_key) or len(inchi_key) == 0 else inchi_key
    cpd.inchi_key = inchi_key
    if cpd.inchi_key:
        a, b, p = cpd.inchi_key.split('-')
        if a not in inchi_key_lookup:
            inchi_key_lookup[a] = {}
        if b not in inchi_key_lookup[a]:
            inchi_key_lookup[a][b] = set()
        inchi_key_lookup[a][b].add((cpd.id, p))

In [6]:
metabolite_reactions = {}
for rxn in modelseed_git.reactions:
    if not rxn.is_obsolete:
        for m in rxn.metabolites:
            if m.seed_id not in metabolite_reactions:
                metabolite_reactions[m.seed_id] = set()
            metabolite_reactions[m.seed_id].add(rxn.id)

In [14]:
import cobrakbase
kbase = cobrakbase.KBaseAPI()
genome = kbase.get_from_ws('87388/3/1')

In [38]:
genome.features.get_by_id('A3776_RS08395').data

{'aliases': [['protein_id', 'WP_066378293.1'], ['locus_tag', 'A3776_RS08395']],
 'cdss': ['A3776_RS08395_CDS_1'],
 'dna_sequence': 'ATGATTGTACCCTTGCTATATCTGGCTTTAGCCGGAGCCTATCTATTAGTTGTCCCGGTGGCTTTAATGCTTTACCTCAACCTACGCTGGTATACGGCTGGCTCGATTGAGCGCACCGTCATGTATTTTTTTGTATTTTTGTTCTTTCCCGGACTGTTGGTTTTGTCGCCGTTTGTGAATCTGCGACCCAAACCCCGCAAAATTGAAGTTTAA',
 'dna_sequence_length': 213,
 'functions': ['NADH dehydrogenase subunit', 'NdhL'],
 'id': 'A3776_RS08395',
 'location': [['NZ_LUHI01000036.1', 21434, '-', 213]],
 'md5': '1ae8b7b3d705b63184c99a9711785d23',
 'protein_md5': '1ae8b7b3d705b63184c99a9711785d23',
 'protein_translation': 'MIVPLLYLALAGAYLLVVPVALMLYLNLRWYTAGSIERTVMYFFVFLFFPGLLVLSPFVNLRPKPRKIEV',
 'protein_translation_length': 70,
 'quality': {'hit_count': 5, 'weighted_hit_count': 5.5205}}

In [221]:
import json
data = None
with open('/home/fliu/workspace/data/sbml/iAnC892/iAnC892_single.json', 'r') as fh:
    data = json.load(fh)

In [222]:
%run /home/fliu/workspace/data/sbml/iAnC892/tools.py
integrate_to_seed(data)

In [161]:
with open('/home/fliu/workspace/data/sbml/iAnC892/iAnC892_single_mapped.json', 'w') as fh:
    fh.write(json.dumps(data, indent=4, sort_keys=True))

In [224]:
model_mapped.metabolites.cpd00011_p

Metabolite identifier,cpd00011_p
Name,CO2
Memory address,0x07fcba66f68e0
Formula,CO2
Compartment,p
In 2 reaction(s),"cpd00011exccc_p, cpd00011excpp_e"


In [225]:
model_mapped = cobra.io.load_json_model('/home/fliu/workspace/data/sbml/iAnC892/iAnC892_single_mapped.json')
from modelseedpy import MSBuilder
from cobra.core import Reaction
MSBuilder.build_exchanges(model_mapped, 'e')
atp_obj = Reaction('obj_ATP', 'ATP objective function', 'objetive function', 0, 1000)
atp_obj.add_metabolites({
    model_mapped.metabolites.cpd00001_c: -1,
    model_mapped.metabolites.cpd00002_c: -1,
    model_mapped.metabolites.cpd00008_c: 1,
    model_mapped.metabolites.cpd00009_c: 1,
    model_mapped.metabolites.cpd00067_c: 1,
})
model_mapped.add_reactions([atp_obj])
model_mapped.objective = 'obj_ATP'
model_mapped.medium = {
    #'EX_cpd00067_e': 1,
    'EX_cpd11632_e': 0.5,
    #'EX_hvphoton2_e': 0.5,
    'EX_hvphoton1_e': 0.5
}
#1.0 obj_ATP = 0.8035714285714285
model_mapped.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
cpd11632_e,EX_cpd11632_e,0.5,0,0.00%
hvphoton1_e,EX_hvphoton1_e,0.5,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux


In [197]:
model_mapped.medium = {
        'EX_cpd00067_e': 1
    }
sol = cobra.flux_analysis.pfba(model_mapped)

In [ ]:
for a in [{'EX_cpd11632_e': 1}, {'EX_hvphoton1_e': 1}]:
    model_mapped.medium = {
        k.id: 1,
    }
    sol = cobra.flux_analysis.pfba(model_mapped)

    print(model_mapped.medium)
    ss = f"{cpd.id}\t{cpd.name}\t{sol.fluxes['obj_ATP']}\t{sol.status}"
    print(ss)

In [ ]:
#glc/O2 22.857142857142854
#glc 0

In [218]:
model_mapped.medium = {
    'EX_cpd00067_e': 1000,
    'EX_cpd00001_e': 1000,
    'EX_cpd00242_e': 1, #H2CO3
    'EX_cpd00528_e': 1000, #N2
    'EX_cpd00011_e': 1000, #co2
    'EX_cpd00007_e': 0, #O2
    'EX_cpd11632_e': 0,
    'EX_hvphoton1_e': 0,
    'EX_cpd00027_e': 1,
    'EX_cpd00048_e': 1,
    'EX_cpd00209_e': 1,
    'EX_cpd11640_e': 1, #H2
    'EX_cpd00268_e': 10
    }
model_mapped.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


In [203]:
for k in model_mapped.exchanges:
    cpd = list(k.metabolites)[0]
    model_mapped.medium = {
        k.id: 1,
    }
    sol = cobra.flux_analysis.pfba(model_mapped)
    ss = f"{cpd.id}\t{cpd.name}\t{sol.fluxes['obj_ATP']}\t{sol.status}"
    print(ss)

cpd11632_e	Photosystem 2 photon	0.8571428571428571	optimal
hvphoton1_e	Photosystem 1 photon	0.75	optimal
cpd00067_e	H+	0.0	optimal
cpd00001_e	H2O	0.0	optimal
cpd00007_e	O2	0.0	optimal
cpd00009_e	Phosphate	0.0	optimal
cpd00048_e	Sulfate	0.0	optimal
cpd00209_e	Nitrate	0.0	optimal
cpd00254_e	Mg	0.0	optimal
cpd10515_e	Fe+2	0.0	optimal
cpd00149_e	Co2+	0.0	optimal
cpd00034_e	Zn2+	0.0	optimal
cpd00058_e	Cu2+	0.0	optimal
cpd20863_e	Mn2+	0.0	optimal
cpd00063_e	Ca2+	0.0	optimal
cpd00205_e	K+	0.0	optimal
cpd00971_e	Na+	0.0	optimal
cpd11574_e	Molybdate	0.0	optimal
cpd00011_e	CO2	0.0	optimal
cpd00528_e	N2	0.0	optimal
cpd00027_e	D-Glucose	0.0	optimal
cpd00053_e	L-Glutamine	0.0	optimal
cpd00107_e	L-Leucine	0.0	optimal
cpd00118_e	Putrescine	0.0	optimal
cpd00129_e	L-Proline	0.0	optimal
cpd00156_e	L-Valine	0.0	optimal
cpd00179_e	Maltose	0.0	optimal
cpd00264_e	Spermidine	0.0	optimal
cpd00268_e	H2S2O3	0.0	optimal
cpd00322_e	L-Isoleucine	0.0	optimal
cpd03725_e	Fe(III)dicitrate	0.0	optimal
cpd00060_e	L-Meth

In [43]:
for k in ['A3776_RS12690', 'A3776_RS16110']:
    for o in genome.features.get_by_id(k).functions:
        print(o)


Dihydroorotate dehydrogenase (quinone) (EC 1.3.5.2)
Putative dihydropyrimidine dehydrogenase [NADP+], similar to dihydroorotate dehydrogenase


In [19]:
for rxn_id in super_c:
    if rxn_id in model.reactions:
        stoich_key, compartment_setup = super_c[rxn_id]
        rxn_model = model.reactions.get_by_id(rxn_id)
        print(rxn_model.id, rxn_model.gene_reaction_rule)

NDH1_1u_k A3776_RS06230 and A3776_RS16180 and A3776_RS20905 and A3776_RS06215 and A3776_RS06220 and A3776_RS19180 and ( A3776_RS06225 or A3776_RS21975 ) and A3776_RS20895 and A3776_RS20900 and A3776_RS08395 and A3776_RS11390 and A3776_RS03245 and A3776_RS09810 and A3776_RS09805 and A3776_RS09800 and ( A3776_RS08060 or A3776_RS10415 or A3776_RS22280 ) and ( A3776_RS03760 or A3776_RS08065 or A3776_RS18720 )
NDH1_1p_p A3776_RS06230 and A3776_RS16180 and A3776_RS20905 and A3776_RS06215 and A3776_RS06220 and A3776_RS19180 and ( A3776_RS06225 or A3776_RS21975 ) and A3776_RS20895 and A3776_RS20900 and A3776_RS08395 and A3776_RS11390 and A3776_RS03245 and A3776_RS09810 and A3776_RS09805 and A3776_RS09800 and ( A3776_RS08060 or A3776_RS10415 or A3776_RS22280 ) and ( A3776_RS03760 or A3776_RS08065 or A3776_RS18720 )
NDH1_4pp_p A3776_RS06230 and A3776_RS16180 and A3776_RS20905 and A3776_RS06215 and A3776_RS06220 and A3776_RS19180 and ( A3776_RS06225 or A3776_RS21975 ) and A3776_RS20895 and A3776_

In [7]:
import cobra
model = cobra.io.load_json_model('/home/fliu/workspace/data/sbml/iAnC892/iAnC892_single.json')

In [64]:
model.reactions.CBFCu_k

Reaction identifier,CBFCu_k
Name,cytochrome b6/f complex
Memory address,0x07fcbbf9f9f70
Stoichiometry,2.0 cpd00067_c + 2.0 cpd12265_k + cpd16503_k --> 4.0 cpd00067_k + cpd07274_k + 2.0 cpd12239_k 2.0 H+ + 2.0 Oxidized plastocyanin + Plastoquinol-9 --> 4.0 H+ + Plastoquinone A + 2.0 Reduced plastocyanin
GPR,A3776_RS13955 or A3776_RS22750
Lower bound,0.0
Upper bound,1000.0


In [62]:
seed_build = {
    'PSII_k': ('rxn46370', {'0': 'k', '1': 'c'}),
}
modelseed_git.reactions.rxn46370.to_template_reaction({'0': 'k', '1': 'c'})

cpd00007_0 0 cpd00007_k
-1.0
cpd00067_0 0 cpd00067_k
-4.0
cpd16503_0 0 cpd16503_k
-2.0
cpd00001_0 0 cpd00001_k
2.0
cpd00067_1 1 cpd00067_c
4.0
cpd07274_0 0 cpd07274_k
2.0
cpd11632_0 0 cpd11632_k
1.0


Reaction identifier,rxn46370_k
Name,nan
Memory address,0x07fcbbc01f790
Stoichiometry,cpd00007_k + 4.0 cpd00067_k + 2.0 cpd16503_k <=> 2.0 cpd00001_k + 4.0 cpd00067_c + 2.0 cpd07274_k + cpd11632_k O2 + 4.0 H+ + 2.0 Plastoquinol-9 <=> 2.0 H2O + 4.0 H+ + 2.0 Plastoquinone A + hn
GPR,
Lower bound,-1000
Upper bound,1000


In [170]:
#rxn37623: cpd00005_0 + 3.0 cpd00067_0 + cpd07274_0 <=> cpd00006_0 + cpd29046_0
#rxn37623 NADPH + 3.0 H+ + Plastoquinone A <=> NADP + plastoquinol A
#rxn36824 NADPH + H+ + Plastoquinone A <=> NADP + Plastoquinol-9
#rxn36824: cpd00005_0 + cpd00067_0 + cpd07274_0 <=> cpd00006_0 + cpd16503_0
super_c = {
    'NDH1_1u_k': ('NDH1_1u_k', {'0': 'c', '1': 'k'}),
    'NDH1_1p_p': ('NDH1_1u_k', {'0': 'c', '1': 'p'}),
    'NDH1_4pp_p': ('NDH1_3u_k', {'0': 'c', '1': 'p'}),
    'NDH1_3u_k': ('NDH1_3u_k', {'0': 'c', '1': 'k'}),
    'NDH1_2p_p': ('NDH1_2u_k', {'0': 'c', '1': 'p'}),
    'NDH1_2u_k': ('NDH1_2u_k', {'0': 'c', '1': 'k'}),
    
    'rxn08335_p_p': ('DHORD_PQ9_k', {'0': 'c', '1': 'p'}),
    'DHORD_PQ9_k': ('DHORD_PQ9_k', {'0': 'c', '1': 'k'}),
    'FQR_k': ('FQR_k', {'0': 'c', '1': 'k'}),
    'CBFCu_k': ('CBFCu_k', {'0': 'c', '1': 'k'}),
}
#           cpd00005_c + 4.0 cpd00067_c + cpd07274_k --> cpd00006_c + 3.0 cpd00067_k + cpd16503_k
#NDH1_1p_p: cpd00005_c + 4.0 cpd00067_c + cpd07274_p --> cpd00006_c + 3.0 cpd00067_p + cpd16503_p
#NDH1_1u_k: cpd00005_c + 4.0 cpd00067_c + cpd07274_k --> cpd00006_c + 3.0 cpd00067_k + cpd16503_k
super_s = {
    'NDH1_1u_k': {
        ('cpd00005', '0'): -1, 
        ('cpd00067', '0'): -4, 
        ('cpd07274', '1'): -1,
        ('cpd00006', '0'): 1, 
        ('cpd00067', '1'): 3, 
        ('cpd16503', '1'): 1
    },

#NDH1_4pp_p: cpd00001_c + cpd00005_c + cpd00011_p + 3.0 cpd00067_c + cpd07274_p --> cpd00006_c + 3.0 cpd00067_p + cpd00242_c + cpd16503_p
#NDH1_3u_k : cpd00001_c + cpd00005_c + cpd00011_p + 3.0 cpd00067_c + cpd07274_k --> cpd00006_c + 3.0 cpd00067_k + cpd00242_c + cpd16503_k
    'NDH1_3u_k': {
        ('cpd00001', '0'): -1, 
        ('cpd00005', '0'): -1, 
        ('cpd00011', '0'): -1,
        ('cpd00067', '0'): -3,
        ('cpd07274', '1'): -1,
        ('cpd00006', '0'): 1, 
        ('cpd00067', '1'): 3, 
        ('cpd00242', '0'): 1,
        ('cpd16503', '1'): 1
    },
#NDH1_2p_p: cpd00004_c + 4.0 cpd00067_c + cpd07274_p --> cpd00003_c + 3.0 cpd00067_p + cpd16503_p
#NDH1_2u_k: cpd00004_c + 4.0 cpd00067_c + cpd07274_k --> cpd00003_c + 3.0 cpd00067_k + cpd16503_k
    'NDH1_2u_k': {
        ('cpd00004', '0'): -1, 
        ('cpd00067', '0'): -4, 
        ('cpd07274', '1'): -1,
        ('cpd00003', '0'): 1,
        ('cpd00067', '1'): 3, 
        ('cpd16503', '1'): 1
    },
#NDH2_syn_k: cpd00004_c + cpd00067_c + cpd07274_k --> cpd00003_c + cpd16503_k
    'NDH2_syn_k': {
        ('cpd00004', 'c'): -1, 
        ('cpd00067', 'c'): -1, 
        ('cpd07274', 'k'): -1,
        ('cpd00003', 'c'): 1,
        ('cpd16503', 'k'): 1
    },
#rxn08335_p_p: cpd00282_c + cpd07274_p --> cpd00247_c + cpd16503_p
#DHORD_PQ9_k : cpd00282_c + cpd07274_k --> cpd00247_c + cpd16503_k
    'DHORD_PQ9_k': {
        ('cpd00282', '0'): -1, 
        ('cpd07274', '1'): -1, 
        ('cpd00247', '0'): 1,
        ('cpd16503', '1'): 1
    },
#FQR_k: 2.0 cpd00067_c + cpd07274_k + 2.0 cpd11620_c --> 2.0 cpd11621_c + cpd16503_k
    'FQR_k': {
        ('cpd00067', '0'): -2, 
        ('cpd07274', '1'): -1, 
        ('cpd11620', '0'): -2,
        ('cpd11621', '0'): 2,
        ('cpd16503', '1'): 1
    },
#CBFCu_k: 2.0 cpd00067_c + 2.0 cpd12265_k + cpd16503_k --> 4.0 cpd00067_k + cpd07274_k + 2.0 cpd12239_k
    'CBFCu_k': {
        ('cpd00067', '0'): -2, 
        ('cpd12265', '1'): -2, 
        ('cpd16503', '1'): -1,
        ('cpd00067', '1'): 4,
        ('cpd07274', '1'): 1,
        ('cpd12239', '1'): 2
    },
}

In [150]:
# CYTBDu_k -> DB with pq9

In [174]:
def test(model_mapped):
    for rxn_id in super_c:
        if rxn_id in model_mapped.reactions:
            stoich_key, compartment_setup = super_c[rxn_id]
            rxn_model = model_mapped.reactions.get_by_id(rxn_id)
            #print(rxn_id, rxn_model)
            #
            rxn_s_model = dict(map(lambda x: (x[0].id, x[1]), rxn_model.metabolites.items()))
            rxn_s_match = {}
            for (cpd_id, cmp_token), v in super_s[stoich_key].items():
                _cpd_id = f'{cpd_id}_{compartment_setup[cmp_token]}'
                rxn_s_match[_cpd_id] = v

            valid = rxn_s_model == rxn_s_match
            print(rxn_id, valid)
            if not valid:
                print(rxn_id, compartment_setup)
                print('rxn_s_match', rxn_s_match)
                print('rxn_s_model', rxn_s_model)
test(model_mapped)

NDH1_1u_k True
NDH1_1p_p True
NDH1_4pp_p True
NDH1_3u_k True
NDH1_2p_p True
NDH1_2u_k True
rxn08335_p_p True
DHORD_PQ9_k True
FQR_k True
CBFCu_k True


In [ ]:
rxn14159: cpd00006_0 + cpd00067_0 + 2.0 cpd11620_0 <=> cpd00005_0 + 2.0 cpd11621_0
rxn14159 NADP + H+ + 2.0 Reducedferredoxin <=> NADPH + 2.0 Oxidizedferredoxin

rxn05937: cpd00006_0 + cpd00067_0 + cpd11620_0 <=> cpd00005_0 + cpd11621_0
rxn05937 NADP + H+ + Reducedferredoxin <=> NADPH + Oxidizedferredoxin

In [226]:
def finddd(a, b):
    c = set(a) | set(b)
    super_match = None
    for i in c:
        if i in metabolite_reactions:
            if super_match is None:
                super_match = set(metabolite_reactions[i])
            super_match &= set(metabolite_reactions[i])
    return super_match

#finddd(['cpd00003'], ['cpd00004'])
#cpd00001_k + 2.0 cpd00067_c + cpd07274_k + 2.0 hvphoton2_c --> 0.5 cpd00007_k + 2.0 cpd00067_k + cpd16503_k
#2 cpd00001_k + 4.0 cpd00067_c + 2 cpd07274_k + 4.0 hvphoton2_c --> 1 cpd00007_k + 4.0 cpd00067_k + 2 cpd16503_k
for r in finddd(['cpd00006', 'cpd11620'], ['cpd00005', 'cpd11621']):
    rxn = modelseed_git.reactions.get_by_id(r)
    print(rxn, rxn.compartments)
    print(rxn.id, rxn.build_reaction_string(True))
    print()

rxn34892: 4.0 cpd00005_0 + cpd00007_0 + cpd11621_0 + cpd30237_0 <=> 2.0 cpd00001_0 + 4.0 cpd00006_0 + cpd11620_0 + cpd30238_0 {'0'}
rxn34892 4.0 NADPH + O2 + Oxidizedferredoxin + Digalactosyldiacylglycerol (1-(9Z)-octadecenoyl,2-(7Z,10Z)-hexadecadienoyl, 18:1(9Z)/16:2(7Z,10Z)) <=> 2.0 H2O + 4.0 NADP + Reducedferredoxin + Digalactosyldiacylglycerol (1-(9Z)-octadecenoyl,2-(4Z,7Z,10Z)-hexadecatrienoyl, 18:1(9Z)/16:3(4Z,7Z,10Z))

rxn34920: 4.0 cpd00005_0 + cpd00007_0 + cpd11621_0 + cpd30244_0 <=> 2.0 cpd00001_0 + 4.0 cpd00006_0 + cpd11620_0 + cpd30248_0 {'0'}
rxn34920 4.0 NADPH + O2 + Oxidizedferredoxin + Digalactosyldiacylglycerol (1-(9Z,12Z)-octadecadienoyl,2-(4Z,7Z,10Z)-hexadecatrienoyl, 18:2(9Z,12Z)/16:3(4Z,7Z,10Z)) <=> 2.0 H2O + 4.0 NADP + Reducedferredoxin + Digalactosyldiacylglycerol (1-(9Z,12Z,15Z)-octadecatrienoyl,2-(4Z,7Z,10Z)-hexadecatrienoyl, 18:3(9Z,12Z,15Z)/16:3(4Z,7Z,10Z))

rxn35092: 4.0 cpd00005_0 + cpd00007_0 + cpd11621_0 + cpd30556_0 <=> 2.0 cpd00001_0 + 4.0 cpd00006_0 + 

In [ ]:
#2 cpd00001_0 + 4.0 cpd00067_1 + 2 cpd07274_0 + cpd11632_0      --> 1 cpd00007_0 + 4.0 cpd00067_0 + 2 cpd16503_0
#2 cpd00001_k + 4.0 cpd00067_c + 2 cpd07274_k + 4.0 hvphoton2_c --> 1 cpd00007_k + 4.0 cpd00067_k + 2 cpd16503_k

In [48]:
modelseed_git.reactions.rxn05937

Reaction identifier,rxn05937
Name,Ferredoxin:NADP+ oxidoreductase
Memory address,0x07fcbbd112c70
Stoichiometry,cpd00006_0 + cpd00067_0 + cpd11620_0 <=> cpd00005_0 + cpd11621_0 NADP + H+ + Reducedferredoxin <=> NADPH + Oxidizedferredoxin
GPR,
Lower bound,-1000
Upper bound,1000


In [ ]:
cpd00006_c + cpd00067_c + 2 cpd11620_c --> cpd00005_c + 2 cpd11621_c